In [12]:
import sys
from pathlib import Path

import pandas as pd

current_dir = Path('.')
current_dir = current_dir.absolute()
root_dir = current_dir.parent.parent
src_dir = root_dir / 'src'

sys.path.append(str(src_dir))

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv(str(src_dir / '.env'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [13]:
from api.containers import analyze_container


analyze = analyze_container.service()
df = analyze.collect_all()
df

F:\Projects\hltv_notebook\src\infra\data_frames\pipeline\matches.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[C.Winner] = winners
F:\Projects\hltv_notebook\src\infra\data_frames\pipeline\matches.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part_df.rename(columns=dict(zip(part, new_columns)), inplace=True)
F:\Projects\hltv_notebook\src\infra\data_frames\pipeline\matches.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

,Team,Score,Map,Opponent,Opponent Score,Date,Event,Is Winner
0,Budapest Five,5,Inferno,Katuna,16,26/2/23,CCT South Europe Series 3,False
1,Katuna,14,Ancient,Budapest Five,16,26/2/23,CCT South Europe Series 3,False
2,Budapest Five,12,Overpass,Katuna,16,26/2/23,CCT South Europe Series 3,False
3,ECSTATIC,14,Inferno,Astralis Talent,16,26/2/23,CCT North Europe Series 4 Closed Qualifier,False
4,Eternal Fire,15,Overpass,fnatic,19,26/2/23,ESL Pro League Season 17,False
...,...,...,...,...,...,...,...,...
50715,Virtus.pro,7,Overpass,Spirit,16,29/6/21,StarLadder CIS RMR 2021,False
50716,Virtus.pro,13,Dust2,Spirit,16,29/6/21,StarLadder CIS RMR 2021,False
50717,Virtus.pro,16,Inferno,Spirit,11,29/6/21,StarLadder CIS RMR 2021,True
50718,Gambit,16,Dust2,FORZE,6,29/6/21,StarLadder CIS RMR 2021,True


In [14]:
class Columns:
    TEAM = 'Team'
    SCORE = 'Score'
    OPPONENT = 'Opponent'
    OPPONENT_SCORE = 'Opponent Score'
    DATE = 'Date'
    IS_WINNER = 'Is Winner'
    MAP = 'Map'
    EVENT = 'Event'

In [15]:
wins = df.groupby([Columns.TEAM, Columns.MAP, Columns.IS_WINNER]).size()
wins = wins.unstack().fillna(0)
wins

Is Winner          False  True 
Team     Map                   
00NATION Ancient    10.0   14.0
         Anubis      0.0    1.0
         Dust2       5.0   11.0
         Inferno    13.0   18.0
         Mirage     19.0   14.0
...                  ...    ...
zu krass Ancient     1.0    1.0
         Inferno     1.0    0.0
         Overpass    0.0    1.0
ГћГіr    Nuke        1.0    0.0
         Overpass    1.0    0.0

[6042 rows x 2 columns]

In [16]:
wins['Total'] = wins[True] + wins[False]
wins['Win Rate'] = wins[True] / wins['Total']
wins

Is Winner          False  True  Total  Win Rate
Team     Map                                   
00NATION Ancient    10.0  14.0   24.0  0.583333
         Anubis      0.0   1.0    1.0  1.000000
         Dust2       5.0  11.0   16.0  0.687500
         Inferno    13.0  18.0   31.0  0.580645
         Mirage     19.0  14.0   33.0  0.424242
...                  ...   ...    ...       ...
zu krass Ancient     1.0   1.0    2.0  0.500000
         Inferno     1.0   0.0    1.0  0.000000
         Overpass    0.0   1.0    1.0  1.000000
ГћГіr    Nuke        1.0   0.0    1.0  0.000000
         Overpass    1.0   0.0    1.0  0.000000

[6042 rows x 4 columns]

In [39]:
wins_vs_any_team = df.groupby([Columns.TEAM, Columns.IS_WINNER]).size()
wins_vs_any_team = wins_vs_any_team.unstack().fillna(0)

wins_vs_any_team['TG'] = wins_vs_any_team[True] + wins_vs_any_team[False]
wins_vs_any_team['WR'] = wins_vs_any_team[True] / wins_vs_any_team['TG']

wins_vs_any_team.rename(columns={True: 'W', False: 'L'}, inplace=True)

# Loses and wins as integers
integer_columns = ['W', 'L', 'TG']


for column in integer_columns:
    wins_vs_any_team[column] = wins_vs_any_team[column].astype(int)

wins_vs_any_team

Is Winner,L,W,TG,WR
Team,,,,
00NATION,89,87,176,0.494318
0to100,21,32,53,0.603774
1 JIN,11,12,23,0.521739
100PG,17,13,30,0.433333
10org,1,0,1,0.000000
...,...,...,...,...
yologang420,2,0,2,0.000000
yur,2,5,7,0.714286
ztrfanclub,0,4,4,1.000000


In [43]:
teams = df.groupby([Columns.TEAM, Columns.OPPONENT, Columns.MAP, Columns.IS_WINNER]).size()
teams = teams.unstack().fillna(0)


# TG = Total Games
teams['TG'] = teams[True] + teams[False]

# TWR = Team Win Rate
teams['TWR'] = teams[True] / teams['TG']


for line in teams.itertuples():
    team, opponent, map = line.Index

    # TTWROM = Total Team Win Rate On Map
    teams.at[line.Index, 'TTWROM'] = wins.at[(team, map), 'Win Rate']

    # TOWROM = Total Opponent Win Rate On Map
    teams.at[line.Index, 'TOWROM'] = wins.at[(opponent, map), 'Win Rate']

teams.rename(columns={True: 'W', False: 'L'}, inplace=True)

# Loses and wins as integers
integer_columns = ['W', 'L', 'TG']


for column in integer_columns:
    teams[column] = teams[column].astype(int)

teams = teams[['W', 'L', 'TG', 'TWR', 'TTWROM', 'TOWROM']]
teams

Is Winner                   W  L  TG  TWR    TTWROM    TOWROM
Team     Opponent Map                                        
00NATION 1WIN     Dust2     0  2   2  0.0  0.687500  0.604167
                  Mirage    1  0   1  1.0  0.424242  0.548387
                  Nuke      0  2   2  0.0  0.366667  0.647887
         4glory   Dust2     1  0   1  1.0  0.687500  0.380952
                  Inferno   1  0   1  1.0  0.580645  0.571429
...                        .. ..  ..  ...       ...       ...
zu krass ZURIA    Overpass  1  0   1  1.0  1.000000  0.500000
         iNation  Ancient   0  1   1  0.0  0.500000  0.555556
                  Inferno   0  1   1  0.0  0.000000  0.761905
ГћГіr    DUSTY    Nuke      0  1   1  0.0  0.000000  0.666667
                  Overpass  0  1   1  0.0  0.000000  0.333333

[39282 rows x 6 columns]

In [54]:
teams.to_csv('grouped_by_maps.csv')

In [53]:
correct_amount = 0
incorrect_amount = 0

for line in teams.itertuples():
    team, opponent, map = line.Index

    if line.TG < 5:
        continue

    if line.W > line.L and line.TTWROM > line.TOWROM:
        correct_amount += 1
    else:
        incorrect_amount += 1

correct_amount, incorrect_amount

(118, 240)

In [52]:
teams.groupby([Columns.TEAM]).mean()

Is Winner,W,L,TG,TWR,TTWROM,TOWROM
Team,,,,,,
00NATION,0.608392,0.622378,1.230769,0.476107,0.490965,0.524547
0to100,0.780488,0.512195,1.292683,0.597561,0.601213,0.482391
1 JIN,0.521739,0.478261,1.000000,0.521739,0.521739,0.434934
100PG,0.448276,0.586207,1.034483,0.413793,0.434483,0.488608
10org,0.000000,1.000000,1.000000,0.000000,0.000000,0.437500
...,...,...,...,...,...,...
yologang420,0.000000,1.000000,1.000000,0.000000,0.000000,0.833333
yur,0.714286,0.285714,1.000000,0.714286,0.714286,0.367038
ztrfanclub,1.000000,0.000000,1.000000,1.000000,1.000000,0.125000
